In [1]:
from langchain_ollama import OllamaEmbeddings

embed = OllamaEmbeddings(model="llama3.2:1b")

In [2]:
input_text = "The meaning of life is 42"
vector = embed.embed_query(input_text)
print(vector[:3])

[-0.0064833546, 0.044403974, 0.017091589]


In [3]:
input_texts = ["Document 1...", "Document 2..."]
vectors = embed.embed_documents(input_texts)
print(len(vectors))
# The first 3 coordinates for the first vector
print(vectors[0][:3])

2
[-0.030706434, 0.033113766, 0.0015737697]


In [5]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_ollama import OllamaEmbeddings

import os

# Initialize Qdrant client
client = QdrantClient(url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"))


def initialize_qdrant():
    client.create_collection(
        collection_name="expenses",
        vectors_config=VectorParams(size=768, distance=Distance.COSINE),
    )


vector_store = QdrantVectorStore(
    client=client,
    collection_name="expenses",
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
)

In [ ]:
results = vector_store.similarity_search(query="545", k=2)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* 545 spent on coffee, description: 545 on coffee mirpur [{'_id': 'c9ea556f-ced7-41ac-b00a-fe4a999bb805', '_collection_name': 'expenses'}]
* 523 spent on wine, description: bd [{'_id': 'bbaf7d63-639f-4290-8db9-d52765a31a0d', '_collection_name': 'expenses'}]


In [18]:
results = vector_store.similarity_search(query="545", k=1)
print([result.page_content for result in results])

['545 spent on coffee, description: 545 on coffee mirpur']
